# About this notebook
The goal of this notebook is to perform docking analysis.

### The general workflow for docking analysis starts with the output .mae files from docking:

1) From each unzipped .mae file, rename the title for each pose to contain a mark/label to indicate which pose it is
    - Using command line script
    
2) Convert from .mae to .pdb in command line

3) Use PyMol to calculate the rmsd of each pose to reference ligand (lowest rmsd is best)

4) Get gscore corresponding to the pose in the pdb and put it in a table with the rmsd and compound ID

### Functions in this notebook (see block #4):

1) get_mae_gscore
    - Input: glob of mae filepaths
    - Output: dataframe of 2 columns containing compound ID and gscore
    
2) align_rms
    - Input: reference and target pdb
    - Output: rms

In [1]:
from pymol import cmd
import glob
import pandas as pd
from os import system
import time

In [2]:
# define string
convert = '/share/apps/schrodinger/suite2021-1/utilities/structconvert '

# glob all mae files into single list
ref_8088 = "/home/aguan/docking/8088_reference-1.pdb"
ref_8091 = "/home/aguan/docking/8091_reference-1.pdb"

tck_8088 = glob.glob('/home/aguan/docking/docking52tcksXP088/glide-dock_XP_JJC8088_52tcks-000*_raw.mae')
tck_8091 = glob.glob('/home/aguan/docking/docking52tcksXP088/glide-dock_XP_JJC8091_52tcks-000*_raw.mae')

In [3]:
# Given a dataframe from get_rms_table
# return a dataframe with 2 columns: compound name and gscore value
def get_mae_gscore(df, in_dir = ''):
    maes = [a.replace(".pdb", ".mae") for a in df["File"]]
    gscore = []
    for mae in maes:
        with open(in_dir+mae, 'r') as f:
            lines = f.readlines()
            bar = []
            gscore_i = []
            header = []
            title_i = []
            # Grab relevant line indexes
            for i in range(len(lines)):
                if "f_m_ct" in lines[i]:
                    header.append(i)
                if ':::' in lines[i]:
                    bar.append(i)
                if "gscore" in lines[i]:
                    gscore_i.append(i)

            # Grab gscore and compound name
            for i in range(len(header)):
                # Index of bar/section separator immediately after current header line
                y = min(a for a in bar if a > header[i])
                offset = 0
                for j in range(header[i], y):
                    if lines[j] == "":
                        offset+=1
                # Index of gscore relative to header line
                x = gscore_i[i] - header[i] + offset
                
                gscore.append(float(lines[x+y]))

                # Index of compound name relative to header line
#                 x = title_i[i] - header[i]

#                 title.append(lines[x+y].replace("\n", "").replace(" ", ""))
            f.close()
                
    df["gscore"] = gscore
    return df

def align_rms(target1, target2):
    cmd.load(target1, "t1")
    cmd.load(target2, "t2")
    cmd.align("t1", "t2")
    _rms = cmd.rms("t1 and resn unk", "t2 and resn unk")
    cmd.delete("all")
    return _rms

def get_rms_table(ref, target, cur_dir=''):
    name = []
    rms = []
    file = []
    for i in range(len(target)):
        name.append(next(open(target[i])).replace("\n","")[10:])
        rms.append(round(align_rms(ref,target[i]),2))
        file.append(target[i].replace(cur_dir,""))
    data = {"Name":name, "File":file, "RMS":rms}
    return pd.DataFrame(data)

def get_dir(input_base):
    splits = input_base.split('/')
    output = ''
    for i in range(len(splits)-1):
        output += splits[i] + '/'
    return output

def split_mae(maes, in_dir=''):
    for i in range(len(maes)):
        outname = "%s52tcks%d.mae"%(in_dir,i)
        system("%s -split-nstructures 1 %s %s" %(convert, maes[i], outname))

def mae_to_pdb(maes):
    for mae in maes:
        system("%s %s %s"%(convert, mae, mae.replace(".mae",".pdb")))
        
def do_analysis(ref, target):
    cur_dir = get_dir(target[0])
    split_mae(target, cur_dir)
    mae_to_pdb(glob.glob(cur_dir+"52tcks*.mae"))
    df = get_rms_table(ref,glob.glob(cur_dir+"*.pdb"), cur_dir)
    df = get_mae_gscore(df, cur_dir)
    return df

In [4]:
tic = time.time()
df_8088 = do_analysis(ref_8088, tck_8088)
duration = time.time() - tic
print(duration)
df_8088 = df_8088.sort_values(by="RMS")
df_8088.to_excel('/home/aguan/docking/docking52tcksXP088/8088_results.xls', index=False)

 PyMOL not running, entering library mode (experimental)


In [1]:
df_8091 = do_analysis(ref_8091, tck_8091)
df_8091 = df_8091.sort_values(by="RMS")
df_8091.to_excel('/home/aguan/docking/docking52tcksXP091/8091_results.xls', index=False)

NameError: name 'do_analysis' is not defined